In [1]:
import importlib
import sys
import torch
import pickle
import os
import pandas as pd

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')

from model.dropout_uncertainty_enc_dec_LSTM.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

# *** This Notebook is not yet functional ***

In [2]:
#load model


file_path_model = '../../notebooks/training_variational_dropout/Helpdesk/Helpdesk_full_grad_norm_new_4layer.pkl'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)

# Load the dataset
file_path_data_set = '../../../encoded_data/test_weytjens//helpdesk_all_5_test.pkl'
helpdesk_test_dataset = torch.load(file_path_data_set, weights_only=False)

Data set categories:  ([('Activity', 16, {'Assign seriousness': 1, 'Closed': 2, 'Create SW anomaly': 3, 'DUPLICATE': 4, 'EOS': 5, 'INVALID': 6, 'Insert ticket': 7, 'RESOLVED': 8, 'Require upgrade': 9, 'Resolve SW anomaly': 10, 'Resolve ticket': 11, 'Schedule intervention': 12, 'Take in charge ticket': 13, 'VERIFIED': 14, 'Wait': 15}), ('Resource', 24, {'EOS': 1, 'Value 1': 2, 'Value 10': 3, 'Value 11': 4, 'Value 12': 5, 'Value 13': 6, 'Value 14': 7, 'Value 15': 8, 'Value 16': 9, 'Value 17': 10, 'Value 18': 11, 'Value 19': 12, 'Value 2': 13, 'Value 20': 14, 'Value 21': 15, 'Value 22': 16, 'Value 3': 17, 'Value 4': 18, 'Value 5': 19, 'Value 6': 20, 'Value 7': 21, 'Value 8': 22, 'Value 9': 23}), ('VariantIndex', 175, {'1.0': 1, '10.0': 2, '100.0': 3, '103.0': 4, '104.0': 5, '107.0': 6, '109.0': 7, '11.0': 8, '110.0': 9, '112.0': 10, '113.0': 11, '114.0': 12, '115.0': 13, '117.0': 14, '118.0': 15, '12.0': 16, '120.0': 17, '122.0': 18, '123.0': 19, '124.0': 20, '125.0': 21, '126.0': 22, '12

In [3]:

import evaluation.probabilistic_evaluation
importlib.reload(evaluation.probabilistic_evaluation)
from evaluation.probabilistic_evaluation import ProbabilisticEvaluation

new_eval = ProbabilisticEvaluation(model=model, 
                                   dataset=helpdesk_test_dataset,
                                   concept_name='Activity',
                                   num_processes=16,
                                   #growing_num_values = [],
                                   growing_num_values = ['case_elapsed_time'],
                                   samples_per_case = 100,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True,
                                   all_cat=['Activity', 'Resource'],
                                   all_num=['case_elapsed_time', 'event_elapsed_time'])



In [5]:
from tqdm.notebook import tqdm

t = 0
b = 0
class_id = 1

case_items = list(new_eval.cases.items())
for i, (case_name, full_case) in tqdm(enumerate(case_items), total=len(new_eval.cases)):
    for j, (prefix_len, prefix, suffix) in enumerate(new_eval._iterate_case(full_case)):
        model.eval()

        predictions, (h, c), seq_len_pred, output_feature_indeces, prefixes = model.forward(prefix, returns_prefix=True)

        target = predictions[0][0]["Activity_mean"][t, b]
        
        model.zero_grad(set_to_none=True)
        target.backward()
        saliency = prefixes.grad.abs()
        
        print(saliency)

  0%|          | 0/916 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 2)

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu, pearsonr

df = results_df.copy()

# 1) Ensure correctness is boolean/int
df["next_activity_correctness"] = df["next_activity_correctness"].astype(bool)
df["correctness_int"] = df["next_activity_correctness"].astype(int)

# 2) Coerce timeaccuracy to numeric (turns non-numeric strings into NaN)
df["next_activity_timeaccuracy"] = pd.to_numeric(df["next_activity_timeaccuracy"], errors="coerce")

# 3) Remove NaN/inf values
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=["next_activity_timeaccuracy", "correctness_int"])

# Masks after cleaning
correct_mask = df["next_activity_correctness"]
incorrect_mask = ~df["next_activity_correctness"]

correct_values = df.loc[correct_mask, "next_activity_timeaccuracy"].astype(float).to_numpy()
incorrect_values = df.loc[incorrect_mask, "next_activity_timeaccuracy"].astype(float).to_numpy()

print("Counts:", correct_values.size, incorrect_values.size)

# ---- A) P-value for correlation (point-biserial == Pearson with 0/1)
r, p_corr = pearsonr(df["next_activity_timeaccuracy"].to_numpy(),
                     df["correctness_int"].to_numpy())
print(f"Correlation r = {r:.6f}, p = {p_corr:.6e}")

# ---- B) P-value for difference in means (Welch t-test)
t_stat, p_t = ttest_ind(correct_values, incorrect_values, equal_var=False)
print(f"Welch t-test: t = {t_stat:.6f}, p = {p_t:.6e}")

# ---- C) Non-parametric alternative (often better for skewed time deltas)
u_stat, p_u = mannwhitneyu(correct_values, incorrect_values, alternative="two-sided")
print(f"Mann–Whitney U: U = {u_stat:.6f}, p = {p_u:.6e}")


Counts: 2295 1028
Correlation r = 0.169720, p = 6.756917e-23
Welch t-test: t = 10.337822, p = 1.727732e-24
Mann–Whitney U: U = 1564346.000000, p = 3.497120e-51


In [ ]:
import torch
from event_log_loader.new_event_log_loader import EventLogDataset

 # eval mode for inference


helpdesk_test_dataset: EventLogDataset = helpdesk_test_dataset

cat_tensors, num_tensors, case_ids = helpdesk_test_dataset.tensor_list


5161

(tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0997e+00,
           3.8361e+00, -2.0086e-08],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  3.8361e+00,
          -2.0086e-08, -2.0086e-08],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -2.0086e-08,
          -2.0086e-08, -2.0086e-08],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -2.0086e-08,
          -2.0086e-08, -2.0086e-08],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -2.0086e-08,
          -2.0086e-08, -2.0086e-08],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -2.0086e-08,
          -2.0086e-08, -2.0086e-08]]),
 tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.2131e+00,
           5.3174e+00,  3.0736e-08],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  5.3174e+00,
           3.0736e-08,  3.0736e-08],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  3.0736e-08,
           3.0736e-08,  3.0736e-08],
         ...,
         [ 0.0000e+00,  0

# Thema: Saliency Maps:


### Grundansatz:
1. Eine simpele (vanilla) gradient based saliency map entweder einem timestep und entweder bzgl. einer activity oder der temporal component
2. Erweiterung auf Integrated Gradient (soll besser sein, da es sich auf einen baseline input bezieht)
3. Dann SHAP und, wenn zu komplex/compute intense für das model, deepSHAP

### Problem: (Ich 1. hab probiert)

- Starke anpassungen im model notwendig -> sehr zeitaufwändig (evtl. kann ich es in simpleren model probieren?)
- Zentrales Problem: "Preprocessing" passiert noch in der inference() und forward() methode